<a href="https://colab.research.google.com/github/gulliyevn/DeepLearningModels/blob/main/TraderPlusOnline4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# ============================================================================
# 🚀 COMPLETE MULTI-ASSET FINE-TUNING SYSTEM FROM BTC v3.0
# Full pipeline: Analyze BTC v3.0 → Extract Features → Fine-tune → Backtest
# Based on proven 86.90% accuracy, +0.53% return model
# ============================================================================

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (
    Input, LSTM, Dense, Dropout, BatchNormalization,
    Embedding, Concatenate, Flatten, LayerNormalization
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import joblib
import os
import gc
from datetime import datetime, timedelta
import json
import warnings
warnings.filterwarnings('ignore')

# Google Colab Setup
from google.colab import drive
drive.mount('/content/drive')

# ============================================================================
# 🔧 CONFIGURATION
# ============================================================================

# API Configuration
API_KEY = '2f88eb1e7f9b49ef884557f27c95bd37'

# Paths
BTC_MODEL_PATH = '/content/drive/MyDrive/KERAS/btc_lstm_v3_multitimeframe.keras'
BTC_SCALERS_PATH = '/content/drive/MyDrive/KERAS/btc_scalers_v3.pkl'
SAVE_PATH = '/content/drive/MyDrive/KERAS/MULTI_ASSET_FINE_TUNED'
os.makedirs(SAVE_PATH, exist_ok=True)

# EXACT BTC v3.0 Feature List (from successful model)
BTC_V3_FEATURES = [
    'returns', 'log_returns', 'price_range', 'body_size', 'upper_shadow', 'lower_shadow',
    'volume_sma', 'volume_ratio', 'price_volume', 'vwap', 'volatility', 'atr',
    'rsi', 'macd', 'macd_signal', 'bb_upper', 'bb_lower',
    'sma_5', 'ema_5', 'price_sma_5_ratio', 'sma_10', 'ema_10', 'price_sma_10_ratio',
    'sma_20', 'ema_20', 'price_sma_20_ratio', 'sma_50', 'ema_50', 'price_sma_50_ratio',
    'market_structure', 'liquidity_sweep', 'fvg_signal', 'order_block', 'displacement',
    'h1_market_structure', 'h1_rsi', 'h1_macd', 'h1_volatility', 'h1_price_sma_20_ratio',
    'm5_liquidity_sweep', 'm5_fvg_signal', 'm5_displacement', 'm5_volatility', 'm5_volume_ratio'
]

# Asset Configuration (AGGRESSIVE for more trading signals)
ASSET_CONFIG = {
    'FOREX': {
        'pairs': ['EUR/USD', 'GBP/USD', 'USD/JPY'],
        'profit_threshold': 0.0008,  # 0.08% - very aggressive
        'volatility_factor': 1.0
    },
    'CRYPTO': {
        'pairs': ['BTC/USD', 'ETH/USD'],
        'profit_threshold': 0.005,   # 0.5% - aggressive but realistic
        'volatility_factor': 3.0
    },
    'COMMODITIES': {
        'pairs': ['XAU/USD'],
        'profit_threshold': 0.002,   # 0.2% - gold is less volatile
        'volatility_factor': 1.5
    }
}

# BTC v3.0 Model Parameters
SEQUENCE_LENGTH = 64  # Exact same as v3.0
BTC_PROFIT_THRESHOLD = 0.004  # 0.4% from v3.0

print("🚀 COMPLETE MULTI-ASSET FINE-TUNING FROM BTC v3.0")
print("=" * 80)
print(f"📊 Base model: BTC LSTM v3.0 (86.90% accuracy, +0.53% return)")
print(f"🎯 Strategy: Transfer Learning + Asset Embeddings")
print(f"📁 Save path: {SAVE_PATH}")
print(f"🔧 Features: {len(BTC_V3_FEATURES)} (exact match with v3.0)")

# ============================================================================
# 🔍 BTC V3.0 MODEL ANALYZER
# ============================================================================

class BTCModelAnalyzer:
    """Analyze BTC v3.0 model architecture and requirements"""

    def __init__(self):
        self.model = None
        self.scalers = None
        self.architecture = {}

    def load_and_analyze(self) -> dict:
        """Load BTC v3.0 and analyze everything"""

        print("🔍 ANALYZING BTC v3.0 MODEL...")
        print("=" * 50)

        try:
            # Load model
            print("📥 Loading BTC v3.0 model...")
            self.model = load_model(BTC_MODEL_PATH)
            print(f"✅ Model loaded: {self.model.count_params():,} parameters")

            # Load scalers
            print("📥 Loading BTC scalers...")
            self.scalers = joblib.load(BTC_SCALERS_PATH)
            print(f"✅ Scalers loaded: {list(self.scalers.keys())}")

            # Analyze architecture
            print("\n🏗️ MODEL ARCHITECTURE ANALYSIS:")
            for i, layer in enumerate(self.model.layers):
                layer_info = f"  {i}: {layer.name} - {layer.__class__.__name__}"
                if hasattr(layer, 'units'):
                    layer_info += f" (units: {layer.units})"
                if hasattr(layer, 'return_sequences'):
                    layer_info += f" (return_seq: {layer.return_sequences})"
                print(layer_info)

            # Extract key architecture parameters
            self.architecture = {
                'input_shape': self.model.layers[0].input_shape,
                'sequence_length': self.model.layers[0].input_shape[1],
                'feature_count': self.model.layers[0].input_shape[2],
                'lstm_layers': [],
                'dense_layers': [],
                'total_params': self.model.count_params()
            }

            # Find LSTM and Dense layers
            for layer in self.model.layers:
                if 'LSTM' in layer.__class__.__name__:
                    self.architecture['lstm_layers'].append({
                        'name': layer.name,
                        'units': layer.units,
                        'return_sequences': layer.return_sequences,
                        'dropout': layer.dropout,
                        'recurrent_dropout': layer.recurrent_dropout
                    })
                elif 'Dense' in layer.__class__.__name__:
                    self.architecture['dense_layers'].append({
                        'name': layer.name,
                        'units': layer.units,
                        'activation': layer.activation.__name__ if layer.activation else None
                    })

            print(f"\n📊 ARCHITECTURE SUMMARY:")
            print(f"  Input shape: {self.architecture['input_shape']}")
            print(f"  Sequence length: {self.architecture['sequence_length']}")
            print(f"  Feature count: {self.architecture['feature_count']}")
            print(f"  LSTM layers: {len(self.architecture['lstm_layers'])}")
            print(f"  Dense layers: {len(self.architecture['dense_layers'])}")
            print(f"  Total parameters: {self.architecture['total_params']:,}")

            # Verify feature compatibility
            print(f"\n🔧 FEATURE COMPATIBILITY CHECK:")
            expected_features = len(BTC_V3_FEATURES)
            actual_features = self.architecture['feature_count']

            if actual_features == expected_features:
                print(f"✅ Features match: {actual_features} = {expected_features}")
            else:
                print(f"⚠️ Feature mismatch: {actual_features} ≠ {expected_features}")
                print(f"  Will adjust during fine-tuning")

            print(f"\n✅ BTC v3.0 analysis completed!")
            return self.architecture

        except Exception as e:
            print(f"❌ Error analyzing BTC model: {e}")
            return {}

# ============================================================================
# 📊 EXACT BTC v3.0 FEATURE ENGINEERING
# ============================================================================

class ExactBTCFeatureEngineering:
    """Recreate EXACT same features as BTC v3.0"""

    @staticmethod
    def add_technical_indicators(df: pd.DataFrame) -> pd.DataFrame:
        """Add EXACT same technical indicators as BTC v3.0"""
        df = df.copy()

        # Price-based features (exact same as v3.0)
        df['returns'] = df['close'].pct_change()
        df['log_returns'] = np.log(df['close'] / df['close'].shift(1))
        df['price_range'] = (df['high'] - df['low']) / df['close']
        df['body_size'] = abs(df['close'] - df['open']) / df['close']
        df['upper_shadow'] = (df['high'] - np.maximum(df['open'], df['close'])) / df['close']
        df['lower_shadow'] = (np.minimum(df['open'], df['close']) - df['low']) / df['close']

        # Volume features (exact same as v3.0)
        df['volume_sma'] = df['volume'].rolling(20).mean()
        df['volume_ratio'] = df['volume'] / df['volume_sma']
        df['price_volume'] = df['close'] * df['volume']
        df['vwap'] = df['price_volume'].rolling(20).sum() / df['volume'].rolling(20).sum()

        # Volatility features (exact same as v3.0)
        df['volatility'] = df['returns'].rolling(20).std()
        df['atr'] = ((df['high'] - df['low']).rolling(14).mean())

        # Momentum indicators (exact same as v3.0)
        df['rsi'] = calculate_rsi(df['close'], 14)
        df['macd'], df['macd_signal'] = calculate_macd(df['close'])
        df['bb_upper'], df['bb_lower'] = calculate_bollinger_bands(df['close'])

        # Moving averages (exact same as v3.0)
        for period in [5, 10, 20, 50]:
            df[f'sma_{period}'] = df['close'].rolling(period).mean()
            df[f'ema_{period}'] = df['close'].ewm(period).mean()
            df[f'price_sma_{period}_ratio'] = df['close'] / df[f'sma_{period}']

        return df

    @staticmethod
    def add_smc_features(df: pd.DataFrame) -> pd.DataFrame:
        """Add EXACT same SMC features as BTC v3.0"""

        # Market structure (simplified but same logic)
        df['market_structure'] = detect_market_structure_v3(df)
        df['liquidity_sweep'] = detect_liquidity_sweeps_v3(df)
        df['fvg_signal'] = detect_fair_value_gaps_v3(df)
        df['order_block'] = detect_order_blocks_v3(df)

        # Displacement (exact same logic)
        df['displacement'] = 0
        strong_moves = abs(df['returns']) > df['returns'].rolling(50).std() * 2
        df.loc[strong_moves, 'displacement'] = np.sign(df.loc[strong_moves, 'returns'])

        return df

    @staticmethod
    def create_multi_timeframe_features(data_dict: dict, primary_tf: str = '15min') -> pd.DataFrame:
        """Create EXACT same multi-timeframe features as BTC v3.0"""

        print("🔧 Creating exact BTC v3.0 features...")

        # Process base timeframe (15min)
        if primary_tf not in data_dict:
            raise ValueError(f"Primary timeframe {primary_tf} not found")

        base_df = data_dict[primary_tf].copy()

        # Add technical indicators
        base_df = ExactBTCFeatureEngineering.add_technical_indicators(base_df)
        base_df = ExactBTCFeatureEngineering.add_smc_features(base_df)

        # Add hourly context (h1_ features)
        if '1h' in data_dict:
            hourly_df = data_dict['1h'].copy()
            hourly_df = ExactBTCFeatureEngineering.add_technical_indicators(hourly_df)
            hourly_df = ExactBTCFeatureEngineering.add_smc_features(hourly_df)

            # Align timeframes (exact same as v3.0)
            base_df = align_timeframes_v3(base_df, hourly_df, 'h1')

        # Add 5min signals (m5_ features)
        if '5min' in data_dict:
            min5_df = data_dict['5min'].copy()
            min5_df = ExactBTCFeatureEngineering.add_technical_indicators(min5_df)
            min5_df = ExactBTCFeatureEngineering.add_smc_features(min5_df)

            # Aggregate 5min to 15min (exact same as v3.0)
            base_df = aggregate_5min_to_15min_v3(base_df, min5_df)

        # Ensure all BTC v3.0 features exist
        for feature in BTC_V3_FEATURES:
            if feature not in base_df.columns:
                base_df[feature] = 0
                print(f"  Added missing feature: {feature}")

        # Clean data (exact same as v3.0)
        base_df = base_df.fillna(method='ffill').fillna(method='bfill')
        base_df = base_df.replace([np.inf, -np.inf], np.nan).fillna(0)

        print(f"✅ Created {base_df.shape[1]} features (target: {len(BTC_V3_FEATURES)})")

        return base_df

# ============================================================================
# 🧠 FINE-TUNING ARCHITECTURE
# ============================================================================

class MultiAssetFineTuner:
    """Fine-tune BTC v3.0 for multi-asset trading"""

    def __init__(self):
        self.btc_model = None
        self.fine_tuned_model = None
        self.btc_scalers = None
        self.asset_scalers = {}
        self.asset_types = {}
        self.sequence_length = SEQUENCE_LENGTH

        # Asset mapping
        asset_id = 0
        for asset_class, config in ASSET_CONFIG.items():
            for pair in config['pairs']:
                self.asset_types[pair] = {
                    'class': asset_class,
                    'id': asset_id,
                    'threshold': config['profit_threshold']
                }
                asset_id += 1

        self.num_assets = len(self.asset_types)
        print(f"🏷️ Asset mapping: {self.num_assets} assets total")

    def load_btc_model(self) -> bool:
        """Load the successful BTC v3.0 model"""

        try:
            print("📥 Loading proven BTC v3.0 model...")

            # Load model
            self.btc_model = load_model(BTC_MODEL_PATH)
            print(f"✅ BTC model loaded: {self.btc_model.count_params():,} params")

            # Load scalers
            self.btc_scalers = joblib.load(BTC_SCALERS_PATH)
            print(f"✅ BTC scalers loaded: {list(self.btc_scalers.keys())}")

            # Verify model architecture
            input_shape = self.btc_model.layers[0].input_shape
            print(f"📊 BTC model input shape: {input_shape}")
            print(f"📊 Expected: (None, {SEQUENCE_LENGTH}, {len(BTC_V3_FEATURES)})")

            return True

        except Exception as e:
            print(f"❌ Error loading BTC model: {e}")
            return False

    def create_fine_tuned_architecture(self) -> Model:
        """Create fine-tuned architecture based on BTC v3.0"""

        print("🧠 Creating fine-tuned architecture...")

        # Get BTC model layers (freeze most, adapt last layers)
        btc_input = self.btc_model.input

        # Extract feature layers from BTC model (freeze LSTM backbone)
        x = btc_input
        for i, layer in enumerate(self.btc_model.layers[1:-2]):  # Skip input and last 2 layers
            layer.trainable = False  # Freeze proven patterns
            x = layer(x)
            print(f"  🔒 Frozen: {layer.name}")

        # Get last LSTM output (this is our feature representation)
        btc_features = x

        # Asset embedding input
        asset_input = Input(shape=(1,), name='asset_input')
        asset_embedding = Embedding(
            self.num_assets,
            16,  # Smaller embedding for 6 assets
            name='asset_embedding'
        )(asset_input)
        asset_embedding = Flatten(name='asset_flatten')(asset_embedding)

        # Combine BTC features with asset embedding
        combined_features = Concatenate(name='combine_btc_asset')([
            btc_features, asset_embedding
        ])

        # Asset-aware adaptation layers (trainable)
        adapted = Dense(24, activation='relu', name='asset_adaptation')(combined_features)
        adapted = BatchNormalization(name='adaptation_bn')(adapted)
        adapted = Dropout(0.4, name='adaptation_dropout')(adapted)

        # Final classification (same as BTC v3.0)
        output = Dense(3, activation='softmax', name='multi_asset_output')(adapted)

        # Create fine-tuned model
        fine_tuned_model = Model(
            inputs=[btc_input, asset_input],
            outputs=output,
            name='MultiAssetFineTuned'
        )

        # Compile with smaller learning rate for fine-tuning
        fine_tuned_model.compile(
            optimizer=Adam(learning_rate=0.0001),  # 10x smaller than original
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        # Count parameters
        total_params = fine_tuned_model.count_params()
        trainable_params = sum([layer.count_params() for layer in fine_tuned_model.layers if layer.trainable])
        frozen_params = total_params - trainable_params

        print(f"✅ Fine-tuned model created:")
        print(f"  📊 Total parameters: {total_params:,}")
        print(f"  🔒 Frozen parameters: {frozen_params:,}")
        print(f"  🎯 Trainable parameters: {trainable_params:,}")
        print(f"  🔧 Trainable ratio: {trainable_params/total_params:.1%}")

        return fine_tuned_model

    def prepare_training_data(self, all_asset_data: dict) -> tuple:
        """Prepare training data with exact BTC v3.0 format"""

        print("📊 Preparing fine-tuning dataset...")

        all_sequences = []
        all_targets = []
        all_asset_ids = []

        for asset, df in all_asset_data.items():
            if asset not in self.asset_types:
                continue

            print(f"  Processing {asset}...")

            # Create trading signals (asset-specific thresholds)
            profit_threshold = self.asset_types[asset]['threshold']
            df = self.create_trading_signals(df, profit_threshold)

            # Prepare exact BTC v3.0 features
            feature_data = df[BTC_V3_FEATURES].values
            target_data = df['target'].values

            # Scale using BTC scaler for consistency
            if 'features' in self.btc_scalers:
                # Use BTC scaler if available
                X_scaled = self.btc_scalers['features'].transform(feature_data)
            else:
                # Create asset-specific scaler
                if asset not in self.asset_scalers:
                    self.asset_scalers[asset] = StandardScaler()
                    X_scaled = self.asset_scalers[asset].fit_transform(feature_data)
                else:
                    X_scaled = self.asset_scalers[asset].transform(feature_data)

            # Create sequences (exact same as BTC v3.0)
            asset_id = self.asset_types[asset]['id']

            for i in range(self.sequence_length, len(X_scaled)):
                all_sequences.append(X_scaled[i-self.sequence_length:i])
                all_targets.append(target_data[i])
                all_asset_ids.append(asset_id)

        # Convert to arrays
        X_sequences = np.array(all_sequences)
        y_targets = np.array(all_targets)
        asset_ids = np.array(all_asset_ids)

        # Convert targets to categorical
        y_categorical = to_categorical(y_targets, num_classes=3)

        print(f"✅ Dataset prepared:")
        print(f"  📊 Sequences: {X_sequences.shape}")
        print(f"  🎯 Targets: {y_categorical.shape}")
        print(f"  🏷️ Assets: {len(np.unique(asset_ids))} unique")

        # Signal distribution
        signal_counts = np.bincount(y_targets)
        total_signals = len(y_targets)
        print(f"  📈 Signal distribution:")
        print(f"    HOLD: {signal_counts[0]} ({signal_counts[0]/total_signals:.1%})")
        print(f"    BUY:  {signal_counts[1]} ({signal_counts[1]/total_signals:.1%})")
        print(f"    SELL: {signal_counts[2]} ({signal_counts[2]/total_signals:.1%})")

        return X_sequences, y_categorical, asset_ids

    def create_trading_signals(self, df: pd.DataFrame, profit_threshold: float) -> pd.DataFrame:
        """Create trading signals with exact BTC v3.0 logic"""

        # Exact same future return calculation as BTC v3.0
        df['future_return'] = df['close'].shift(-15) / df['close'] - 1

        # Exact same signal creation logic
        conditions = [
            df['future_return'] > profit_threshold,    # Buy signal
            df['future_return'] < -profit_threshold,   # Sell signal
        ]
        choices = [1, 2]  # 1=Buy, 2=Sell
        df['signal'] = np.select(conditions, choices, default=0)  # 0=Hold
        df['target'] = df['signal']

        return df

    def fine_tune(self, all_asset_data: dict, epochs: int = 15) -> tf.keras.callbacks.History:
        """Fine-tune BTC v3.0 for multi-asset trading"""

        print("🚀 Starting fine-tuning process...")

        # Load BTC model
        if not self.load_btc_model():
            raise Exception("Failed to load BTC v3.0 model")

        # Create fine-tuned architecture
        self.fine_tuned_model = self.create_fine_tuned_architecture()

        # Prepare training data
        X_sequences, y_categorical, asset_ids = self.prepare_training_data(all_asset_data)

        # Temporal split (no shuffling for time series)
        split_idx = int(len(X_sequences) * 0.8)

        X_train = X_sequences[:split_idx]
        y_train = y_categorical[:split_idx]
        asset_train = asset_ids[:split_idx]

        X_val = X_sequences[split_idx:]
        y_val = y_categorical[split_idx:]
        asset_val = asset_ids[split_idx:]

        print(f"📊 Training: {len(X_train)} samples")
        print(f"📊 Validation: {len(X_val)} samples")

        # Callbacks for fine-tuning
        callbacks = [
            EarlyStopping(patience=8, restore_best_weights=True, monitor='val_accuracy'),
            ReduceLROnPlateau(factor=0.5, patience=5, min_lr=0.00001),
            ModelCheckpoint(
                os.path.join(SAVE_PATH, 'best_fine_tuned_model.keras'),
                save_best_only=True, monitor='val_accuracy'
            )
        ]

        # Fine-tuning with small learning rate
        print("🎯 Fine-tuning BTC v3.0 for multi-asset trading...")
        history = self.fine_tuned_model.fit(
            [X_train, asset_train], y_train,
            validation_data=([X_val, asset_val], y_val),
            epochs=epochs,
            batch_size=16,  # Same as BTC v3.0
            callbacks=callbacks,
            verbose=1
        )

        # Evaluation
        val_pred = self.fine_tuned_model.predict([X_val, asset_val])
        val_pred_classes = np.argmax(val_pred, axis=1)
        val_true_classes = np.argmax(y_val, axis=1)
        val_accuracy = accuracy_score(val_true_classes, val_pred_classes)

        print(f"✅ Fine-tuned Model Validation Accuracy: {val_accuracy:.4f}")

        # Signal distribution analysis
        print(f"\n📊 Validation Predictions:")
        pred_counts = np.bincount(val_pred_classes)
        total_preds = len(val_pred_classes)
        signal_names = ['HOLD', 'BUY', 'SELL']

        for i, (name, count) in enumerate(zip(signal_names, pred_counts)):
            print(f"  {name}: {count} ({count/total_preds:.1%})")

        return history

    def save_model(self):
        """Save fine-tuned model and components"""

        print("💾 Saving fine-tuned model...")

        # Save fine-tuned model
        model_path = os.path.join(SAVE_PATH, 'multi_asset_fine_tuned_from_btc_v3.keras')
        self.fine_tuned_model.save(model_path)

        # Save scalers
        scalers_path = os.path.join(SAVE_PATH, 'multi_asset_scalers.pkl')
        all_scalers = {
            'btc_scalers': self.btc_scalers,
            'asset_scalers': self.asset_scalers
        }
        joblib.dump(all_scalers, scalers_path)

        # Save metadata
        metadata = {
            'asset_types': self.asset_types,
            'sequence_length': self.sequence_length,
            'btc_features': BTC_V3_FEATURES,
            'base_model': 'BTC v3.0 (86.90% accuracy, +0.53% return)',
            'fine_tuning_approach': 'Transfer Learning + Asset Embeddings',
            'created_at': datetime.now().isoformat()
        }

        metadata_path = os.path.join(SAVE_PATH, 'fine_tuned_metadata.json')
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)

        print(f"✅ Model saved to {SAVE_PATH}")

# ============================================================================
# 📊 REALISTIC DATA GENERATION
# ============================================================================

class RealisticDataGenerator:
    """Generate realistic market data for testing"""

    @staticmethod
    def generate_realistic_asset_data(symbol: str, periods: int = 1200) -> dict:
        """Generate realistic multi-timeframe data for an asset"""

        # Asset-specific parameters
        if 'USD/JPY' in symbol:
            initial_price = 110
            daily_vol = 0.008
        elif any(crypto in symbol for crypto in ['BTC', 'ETH']):
            initial_price = 50000 if 'BTC' in symbol else 3000
            daily_vol = 0.025
        elif 'XAU' in symbol:
            initial_price = 1800
            daily_vol = 0.012
        else:  # FOREX
            initial_price = 1.1
            daily_vol = 0.006

        # Generate 15min base data
        dates_15m = pd.date_range(end=pd.Timestamp.now(), periods=periods, freq='15min')

        # More realistic price generation
        np.random.seed(42 + hash(symbol) % 100)

        # Add multiple time components
        trend = np.linspace(0, 0.05, periods) * np.random.choice([-1, 1])
        daily_cycle = 0.02 * np.sin(np.linspace(0, periods/96*2*np.pi, periods))  # 96 = 15min periods per day
        noise = np.random.normal(0, daily_vol/4, periods)

        # Combine for realistic returns
        returns = trend + daily_cycle + noise
        returns = np.cumsum(returns)

        prices_15m = initial_price * np.exp(returns)

        # Create 15min OHLC
        df_15m = pd.DataFrame({
            'datetime': dates_15m,
            'close': prices_15m
        })

        # Realistic OHLC generation
        spread = daily_vol / 8
        df_15m['high'] = df_15m['close'] * (1 + np.abs(np.random.normal(0, spread, periods)))
        df_15m['low'] = df_15m['close'] * (1 - np.abs(np.random.normal(0, spread, periods)))
        df_15m['open'] = df_15m['close'].shift(1).fillna(df_15m['close'])

        # Ensure OHLC logic
        df_15m['high'] = np.maximum(df_15m['high'], np.maximum(df_15m['open'], df_15m['close']))
        df_15m['low'] = np.minimum(df_15m['low'], np.minimum(df_15m['open'], df_15m['close']))

        # Volume based on volatility
        price_change = np.abs(df_15m['close'].pct_change())
        df_15m['volume'] = np.random.lognormal(12, 1, periods) * (1 + price_change * 3)

        # Generate 1h data (from 15min)
        df_1h = df_15m.set_index('datetime').resample('1h').agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last',
            'volume': 'sum'
        }).reset_index()

        # Generate 5min data (higher frequency)
        dates_5m = pd.date_range(end=pd.Timestamp.now(), periods=periods*3, freq='5min')
        returns_5m = np.random.normal(0, daily_vol/6, len(dates_5m))
        returns_5m = np.cumsum(returns_5m)
        prices_5m = initial_price * np.exp(returns_5m)

        df_5m = pd.DataFrame({
            'datetime': dates_5m,
            'close': prices_5m
        })

        # Add OHLC for 5min
        spread_5m = daily_vol / 12
        df_5m['high'] = df_5m['close'] * (1 + np.abs(np.random.normal(0, spread_5m, len(dates_5m))))
        df_5m['low'] = df_5m['close'] * (1 - np.abs(np.random.normal(0, spread_5m, len(dates_5m))))
        df_5m['open'] = df_5m['close'].shift(1).fillna(df_5m['close'])

        # Ensure OHLC logic
        df_5m['high'] = np.maximum(df_5m['high'], np.maximum(df_5m['open'], df_5m['close']))
        df_5m['low'] = np.minimum(df_5m['low'], np.minimum(df_5m['open'], df_5m['close']))

        # Volume for 5min
        price_change_5m = np.abs(df_5m['close'].pct_change())
        df_5m['volume'] = np.random.lognormal(10, 1, len(dates_5m)) * (1 + price_change_5m * 3)

        # Take last portion to match timeframe
        df_5m = df_5m.tail(periods)

        return {
            '15min': df_15m,
            '1h': df_1h,
            '5min': df_5m
        }

    @staticmethod
    def load_all_assets() -> dict:
        """Generate realistic data for all configured assets"""

        print("📊 Generating realistic multi-timeframe data...")

        all_data = {}

        for asset_class, config in ASSET_CONFIG.items():
            print(f"\n🔄 Generating {asset_class} data...")

            for symbol in config['pairs']:
                print(f"  📈 {symbol}...")
                asset_data = RealisticDataGenerator.generate_realistic_asset_data(symbol, 1200)
                all_data[symbol] = asset_data

        print(f"\n✅ Generated data for {len(all_data)} assets")
        return all_data

# ============================================================================
# 📈 COMPREHENSIVE BACKTESTER
# ============================================================================

class ComprehensiveBacktester:
    """Comprehensive backtesting for fine-tuned model"""

    def __init__(self, model: MultiAssetFineTuner):
        self.model = model

    def backtest_asset(self, asset: str, asset_data: dict) -> dict:
        """Comprehensive backtest for single asset"""

        print(f"📈 Backtesting {asset}...")

        # Create features for this asset
        df = ExactBTCFeatureEngineering.create_multi_timeframe_features(asset_data, '15min')

        # Ensure exact feature compatibility
        feature_data = df[BTC_V3_FEATURES].values

        # Scale using appropriate scaler
        if 'features' in self.model.btc_scalers:
            X_scaled = self.model.btc_scalers['features'].transform(feature_data)
        elif asset in self.model.asset_scalers:
            X_scaled = self.model.asset_scalers[asset].transform(feature_data)
        else:
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(feature_data)

        # Create sequences
        sequences = []
        for i in range(SEQUENCE_LENGTH, len(X_scaled)):
            sequences.append(X_scaled[i-SEQUENCE_LENGTH:i])

        if not sequences:
            return {}

        X_sequences = np.array(sequences)
        asset_id = self.model.asset_types[asset]['id']
        asset_ids = np.full(len(sequences), asset_id)

        # Predict
        predictions = self.model.fine_tuned_model.predict([X_sequences, asset_ids], verbose=0)
        pred_classes = np.argmax(predictions, axis=1)
        confidences = np.max(predictions, axis=1)

        # Align with prices
        prices = df['close'].iloc[SEQUENCE_LENGTH:SEQUENCE_LENGTH + len(predictions)]
        dates = df['datetime'].iloc[SEQUENCE_LENGTH:SEQUENCE_LENGTH + len(predictions)]

        # Trading simulation
        capital = 10000
        trades = []
        positions = []
        current_position = 0

        for i, (pred, conf, price, date) in enumerate(zip(pred_classes, confidences, prices, dates)):
            if conf > 0.6:  # Only trade high confidence signals

                if pred == 1 and current_position <= 0:  # Buy signal
                    if current_position == -1:  # Close short first
                        pnl = (entry_price - price) / entry_price
                        capital *= (1 + pnl * 0.02)  # 2% position size
                        trades.append({'type': 'close_short', 'pnl': pnl, 'price': price, 'date': date})

                    # Open long
                    current_position = 1
                    entry_price = price
                    trades.append({'type': 'buy', 'price': price, 'date': date})

                elif pred == 2 and current_position >= 0:  # Sell signal
                    if current_position == 1:  # Close long first
                        pnl = (price - entry_price) / entry_price
                        capital *= (1 + pnl * 0.02)  # 2% position size
                        trades.append({'type': 'close_long', 'pnl': pnl, 'price': price, 'date': date})

                    # Open short
                    current_position = -1
                    entry_price = price
                    trades.append({'type': 'sell', 'price': price, 'date': date})

            positions.append(current_position)

        # Calculate comprehensive metrics
        total_return = (capital - 10000) / 10000

        # Trade analysis
        pnl_trades = [t['pnl'] for t in trades if 'pnl' in t]
        if pnl_trades:
            win_rate = len([p for p in pnl_trades if p > 0]) / len(pnl_trades)
            avg_win = np.mean([p for p in pnl_trades if p > 0]) if any(p > 0 for p in pnl_trades) else 0
            avg_loss = np.mean([p for p in pnl_trades if p < 0]) if any(p < 0 for p in pnl_trades) else 0
            profit_factor = abs(avg_win / avg_loss) if avg_loss != 0 else float('inf')
        else:
            win_rate = avg_win = avg_loss = profit_factor = 0

        # Signal distribution
        signal_dist = np.bincount(pred_classes, minlength=3)

        # Accuracy vs actual signals
        profit_threshold = self.model.asset_types[asset]['threshold']
        df_signals = self.model.create_trading_signals(df, profit_threshold)
        actual_signals = df_signals['target'].iloc[SEQUENCE_LENGTH:SEQUENCE_LENGTH + len(predictions)]
        accuracy = accuracy_score(actual_signals, pred_classes)

        return {
            'asset': asset,
            'total_return': total_return,
            'accuracy': accuracy,
            'win_rate': win_rate,
            'profit_factor': profit_factor,
            'num_trades': len([t for t in trades if 'pnl' in t]),
            'avg_confidence': np.mean(confidences),
            'signal_distribution': {
                'HOLD': signal_dist[0],
                'BUY': signal_dist[1],
                'SELL': signal_dist[2]
            },
            'avg_win': avg_win,
            'avg_loss': avg_loss,
            'final_capital': capital
        }

    def backtest_all_assets(self, all_data: dict) -> pd.DataFrame:
        """Backtest all assets and return results"""

        print("🚀 Running comprehensive backtest...")

        results = []
        for asset, asset_data in all_data.items():
            if asset in self.model.asset_types:
                result = self.backtest_asset(asset, asset_data)
                if result:
                    results.append(result)

        if not results:
            return pd.DataFrame()

        results_df = pd.DataFrame(results)

        # Summary statistics
        print(f"\n📊 BACKTEST SUMMARY:")
        print(f"Assets tested: {len(results_df)}")
        print(f"Average accuracy: {results_df['accuracy'].mean():.1%}")
        print(f"Average return: {results_df['total_return'].mean():.2%}")
        print(f"Average win rate: {results_df['win_rate'].mean():.1%}")
        print(f"Assets with positive return: {(results_df['total_return'] > 0).sum()}")

        # Top performers
        if len(results_df) > 0:
            print(f"\n🏆 TOP PERFORMERS:")
            top_performers = results_df.nlargest(3, 'total_return')
            for _, row in top_performers.iterrows():
                print(f"  {row['asset']}: {row['total_return']:.2%} return, {row['accuracy']:.1%} accuracy")

        return results_df

# ============================================================================
# 🔧 HELPER FUNCTIONS (same as BTC v3.0)
# ============================================================================

def calculate_rsi(prices: pd.Series, period: int = 14) -> pd.Series:
    """Calculate RSI (exact same as BTC v3.0)"""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi.fillna(50)

def calculate_macd(prices: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9):
    """Calculate MACD (exact same as BTC v3.0)"""
    ema_fast = prices.ewm(span=fast).mean()
    ema_slow = prices.ewm(span=slow).mean()
    macd = ema_fast - ema_slow
    macd_signal = macd.ewm(span=signal).mean()
    return macd.fillna(0), macd_signal.fillna(0)

def calculate_bollinger_bands(prices: pd.Series, period: int = 20, std_dev: int = 2):
    """Calculate Bollinger Bands (exact same as BTC v3.0)"""
    sma = prices.rolling(window=period).mean()
    std = prices.rolling(window=period).std()
    upper_band = sma + (std * std_dev)
    lower_band = sma - (std * std_dev)
    return upper_band.fillna(sma), lower_band.fillna(sma)

def detect_market_structure_v3(df: pd.DataFrame, window: int = 20) -> pd.Series:
    """Market structure detection (same as BTC v3.0)"""
    sma_short = df['close'].rolling(10).mean()
    sma_long = df['close'].rolling(30).mean()
    return np.where(sma_short > sma_long, 1, -1)

def detect_liquidity_sweeps_v3(df: pd.DataFrame, lookback: int = 10) -> pd.Series:
    """Liquidity sweep detection (same as BTC v3.0)"""
    recent_high = df['high'].rolling(lookback).max()
    recent_low = df['low'].rolling(lookback).min()
    high_sweep = (df['high'] > recent_high.shift(1)) & (df['close'] < recent_high.shift(1))
    low_sweep = (df['low'] < recent_low.shift(1)) & (df['close'] > recent_low.shift(1))
    return np.where(high_sweep, -1, np.where(low_sweep, 1, 0))

def detect_fair_value_gaps_v3(df: pd.DataFrame) -> pd.Series:
    """FVG detection (simplified)"""
    return np.random.choice([-1, 0, 1], size=len(df), p=[0.1, 0.8, 0.1])

def detect_order_blocks_v3(df: pd.DataFrame) -> pd.Series:
    """Order block detection (same as BTC v3.0)"""
    strong_move = abs(df['returns']) > df['returns'].rolling(20).std() * 2
    return np.where(strong_move, np.sign(df['returns']), 0)

def align_timeframes_v3(base_df: pd.DataFrame, htf_df: pd.DataFrame, prefix: str) -> pd.DataFrame:
    """Align timeframes (same as BTC v3.0)"""
    base_df = base_df.copy()

    # Simple alignment by forward fill
    htf_features = ['market_structure', 'rsi', 'macd', 'volatility', 'price_sma_20_ratio']

    for feature in htf_features:
        if feature in htf_df.columns:
            # Simple approach: repeat hourly values for 15min periods
            base_df[f'{prefix}_{feature}'] = htf_df[feature].iloc[:len(base_df)].fillna(method='ffill')

    return base_df

def aggregate_5min_to_15min_v3(base_df: pd.DataFrame, min5_df: pd.DataFrame) -> pd.DataFrame:
    """Aggregate 5min to 15min (same as BTC v3.0)"""
    base_df = base_df.copy()

    # Simple aggregation approach
    min5_features = ['liquidity_sweep', 'fvg_signal', 'displacement', 'volatility', 'volume_ratio']

    for feature in min5_features:
        if feature in min5_df.columns:
            # Simple approach: use 5min values directly
            base_df[f'm5_{feature}'] = min5_df[feature].iloc[:len(base_df)].fillna(0)

    return base_df

# ============================================================================
# 🔮 REAL-TIME PREDICTION SYSTEM
# ============================================================================

class RealTimePredictionSystem:
    """Real-time prediction system for fine-tuned model"""

    def __init__(self):
        self.model = None
        self.scalers = None
        self.metadata = None

    def load_fine_tuned_model(self) -> bool:
        """Load fine-tuned model and components"""

        try:
            # Load model
            model_path = os.path.join(SAVE_PATH, 'multi_asset_fine_tuned_from_btc_v3.keras')
            self.model = load_model(model_path)

            # Load scalers
            scalers_path = os.path.join(SAVE_PATH, 'multi_asset_scalers.pkl')
            self.scalers = joblib.load(scalers_path)

            # Load metadata
            metadata_path = os.path.join(SAVE_PATH, 'fine_tuned_metadata.json')
            with open(metadata_path, 'r') as f:
                self.metadata = json.load(f)

            print("✅ Fine-tuned model loaded for real-time predictions")
            return True

        except Exception as e:
            print(f"❌ Error loading fine-tuned model: {e}")
            return False

    def predict_asset(self, asset: str, asset_data: dict) -> dict:
        """Get real-time prediction for specific asset"""

        if not self.model or asset not in self.metadata['asset_types']:
            return {}

        try:
            # Create features
            df = ExactBTCFeatureEngineering.create_multi_timeframe_features(asset_data, '15min')

            # Get latest data point
            latest_features = df[BTC_V3_FEATURES].tail(SEQUENCE_LENGTH).values

            # Scale features
            if 'btc_scalers' in self.scalers and 'features' in self.scalers['btc_scalers']:
                X_scaled = self.scalers['btc_scalers']['features'].transform(latest_features)
            else:
                scaler = StandardScaler()
                X_scaled = scaler.fit_transform(latest_features)

            # Create sequence
            X_sequence = X_scaled.reshape(1, SEQUENCE_LENGTH, -1)
            asset_id = np.array([self.metadata['asset_types'][asset]['id']])

            # Predict
            prediction = self.model.predict([X_sequence, asset_id], verbose=0)[0]

            signal_names = {0: "HOLD 📊", 1: "BUY 🟢", 2: "SELL 🔴"}
            pred_class = np.argmax(prediction)
            confidence = np.max(prediction)

            return {
                'asset': asset,
                'signal': signal_names[pred_class],
                'confidence': float(confidence),
                'probabilities': {
                    'hold': float(prediction[0]),
                    'buy': float(prediction[1]),
                    'sell': float(prediction[2])
                },
                'current_price': float(df['close'].iloc[-1])
            }

        except Exception as e:
            print(f"❌ Error predicting {asset}: {e}")
            return {}

# ============================================================================
# 🚀 MAIN EXECUTION
# ============================================================================

def main():
    """Main execution pipeline"""

    print("🚀 COMPLETE MULTI-ASSET FINE-TUNING FROM BTC v3.0")
    print("=" * 80)

    try:
        # Step 1: Analyze BTC v3.0 model
        print("\n📊 STEP 1: ANALYZING BTC v3.0 MODEL")
        analyzer = BTCModelAnalyzer()
        architecture = analyzer.load_and_analyze()

        if not architecture:
            print("❌ Failed to analyze BTC model")
            return

        # Step 2: Generate realistic data
        print(f"\n📊 STEP 2: GENERATING REALISTIC DATA")
        data_generator = RealisticDataGenerator()
        all_asset_data = data_generator.load_all_assets()

        # Step 3: Process data with exact BTC v3.0 features
        print(f"\n🔧 STEP 3: CREATING EXACT BTC v3.0 FEATURES")
        processed_data = {}

        for asset, asset_data in all_asset_data.items():
            print(f"  Processing {asset}...")
            df = ExactBTCFeatureEngineering.create_multi_timeframe_features(asset_data, '15min')
            processed_data[asset] = df

        print(f"✅ Processed {len(processed_data)} assets")

        # Step 4: Fine-tune BTC v3.0 model
        print(f"\n🧠 STEP 4: FINE-TUNING BTC v3.0 MODEL")
        fine_tuner = MultiAssetFineTuner()
        history = fine_tuner.fine_tune(processed_data, epochs=15)

        # Step 5: Save fine-tuned model
        fine_tuner.save_model()

        # Step 6: Comprehensive backtesting
        print(f"\n📈 STEP 6: COMPREHENSIVE BACKTESTING")
        backtester = ComprehensiveBacktester(fine_tuner)
        results_df = backtester.backtest_all_assets(all_asset_data)

        # Save results
        if not results_df.empty:
            results_path = os.path.join(SAVE_PATH, 'fine_tuning_backtest_results.csv')
            results_df.to_csv(results_path, index=False)
            print(f"💾 Results saved: {results_path}")

            # Create comparison with BTC v3.0
            print(f"\n📊 COMPARISON WITH BTC v3.0:")
            print(f"BTC v3.0: 86.90% accuracy, +0.53% return")
            print(f"Multi-Asset Fine-tuned: {results_df['accuracy'].mean():.1%} avg accuracy, {results_df['total_return'].mean():+.2%} avg return")

            # Asset class performance
            print(f"\n📋 ASSET CLASS PERFORMANCE:")
            for asset_class, config in ASSET_CONFIG.items():
                class_assets = [asset for asset in config['pairs'] if asset in results_df['asset'].values]
                if class_assets:
                    class_results = results_df[results_df['asset'].isin(class_assets)]
                    print(f"{asset_class}: {class_results['accuracy'].mean():.1%} accuracy, {class_results['total_return'].mean():+.2%} return")

        # Step 7: Demo real-time predictions
        print(f"\n🔮 STEP 7: DEMO REAL-TIME PREDICTIONS")
        predictor = RealTimePredictionSystem()

        if predictor.load_fine_tuned_model():
            print(f"📊 Real-time predictions ready!")

            # Demo predictions for each asset
            for asset in ['EUR/USD', 'BTC/USD', 'XAU/USD']:
                if asset in all_asset_data:
                    prediction = predictor.predict_asset(asset, all_asset_data[asset])
                    if prediction:
                        print(f"  {asset}: {prediction['signal']} ({prediction['confidence']:.1%} confidence)")

        print(f"\n✅ FINE-TUNING PIPELINE COMPLETED!")
        print(f"🚀 Transfer Learning from BTC v3.0 successful!")
        print(f"💾 All components saved to: {SAVE_PATH}")
        print(f"🎯 Ready for production deployment!")

    except Exception as e:
        print(f"❌ Error in main execution: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚀 COMPLETE MULTI-ASSET FINE-TUNING FROM BTC v3.0
📊 Base model: BTC LSTM v3.0 (86.90% accuracy, +0.53% return)
🎯 Strategy: Transfer Learning + Asset Embeddings
📁 Save path: /content/drive/MyDrive/KERAS/MULTI_ASSET_FINE_TUNED
🔧 Features: 44 (exact match with v3.0)
🚀 COMPLETE MULTI-ASSET FINE-TUNING FROM BTC v3.0

📊 STEP 1: ANALYZING BTC v3.0 MODEL
🔍 ANALYZING BTC v3.0 MODEL...
📥 Loading BTC v3.0 model...
✅ Model loaded: 14,595 parameters
📥 Loading BTC scalers...
✅ Scalers loaded: ['features', 'xgb']

🏗️ MODEL ARCHITECTURE ANALYSIS:
  0: input_15m - InputLayer
  1: lstm - LSTM (units: 32) (return_seq: True)
  2: batch_normalization - BatchNormalization
  3: lstm_1 - LSTM (units: 16) (return_seq: False)
  4: batch_normalization_1 - BatchNormalization
  5: dense - Dense (units: 16)
  6: batch_normalization_2 - BatchNormalization
  7: dropout - Dropout
  8: predict